# Reaction path analysis prototype

## Objective

We want to sample reaction pathways (and specifically sample around transition-states) in an automated fashion. This means that we need to:

1. Enumerate reactions to search for
2. Identify and/or generate reactants and products
3. Turn those individual reactants and products into entrance and exit complexes
4. From the entrance and exit complexes, generate a guess reaction pathway and/or guess transition-state structure
5. Optimize the transition-state
6. From the transition-state, optimize the reaction endpoints

Right now, I (@espottesmith) don't have an amazing way to enumerate reactions (work in progress), so for this notebook, we'll assume that we have a set of reasonable reactions with reactants and products

In [7]:
# stdlib
from pathlib import Path
from typing import Any, List, Dict, Optional, Union

# Basic numeric/scientific python libraries
import numpy as np
import pandas as pd
import networkx as nx

# For reaction complex formation
from architector import view_structures, convert_io_molecule
import architector.io_arch_dock as io_arch_dock

# For defining a reaction path geodesic using internal coordinates
import geodesic_interpolate

# For ORCA calculations with Sella optimizer
from ase.build import molecule
from ase.io import read
from ase import Atoms
from sella import Sella
from quacc.recipes.orca.core import ase_relax_job

import jobflow as jf
from jobflow.managers.fireworks import flow_to_workflow
from fireworks import LaunchPad

# Unsure if these are necessary - would be nice if we can just stay in ASE world, at least mostly
from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import metal_edge_extender, OpenBabelNN
from pymatgen.io.ase import AseAtomsAdaptor

## Forming complexes

We consider four cases of different levels of difficulty: 
1. One reactant, one product, one reaction center (from DOI: 10.1039/D2DD00117A)
2. Two reactants, one product, one reaction center (from DOI: 10.1021/acs.jpclett.3c03279)
3. Two reactants, two products, two reaction centers (from DOI: 10.1021/acs.jpclett.3c03279)
4. One reactant, three products, two reaction centers, AND with flexible coordination environment (from DOI: 10.1021/acsenergylett.2c02351)

In [5]:
def make_complexes(
    reactants: List[Union[Molecule, Atoms]],
    products: List[Union[Molecule, Atoms]],
    mapping: Dict[Tuple[int, int[, Tuple[int, int]],
    reacting_atoms_reactants: Dict[int, List[int]],
    reacting_atoms_products: Dict[int, List[int]],
    number_complexes: int=1,
):

    # Convert reactants and products to ase Atoms objects
    rcts = [AseAtomsAdaptor.get_atoms(r) if isinstance(r, Molecule) else r for r in reactants]
    pros = [AseAtomsAdaptor.get_atoms(p) if isinstance(p, Molecule) else p for p in products]

    entrance_complexes = list()
    exit_complexes = list()
    
    if len(rcts) == 1:
        # For now, don't bother re-posing the reactant if there's only one
        # TODO: Probably we can use CREST to get a conformer that's closest to the product(s)?
        entrance_complexes = rcts
    else:
        # Pick one molecule to be the center
        # Arbitrarily, we choose the molecule with the most reacting atoms
        # If there's a tie, we choose the one that's largest
        central_index = None
        central_mol = None
        for ii, rct in enumerate(reactants):
            reacting_atoms = reacting_atoms_reactants.get(ii)
            if reacting_atoms is None:
                raise ValueError("Need to provide list of reacting atoms for all reactants!"
                                 "Format of reacting_atoms_reactants: {i: [ind_1, ind_2,..., ind_n]},"
                                 "where 'i' is the 0-based index of the reactant in `reactants`"
                                 "and 'ind_n' is the nth reacting atom, again using 0-based indices")

            if central_index is None:
                central_index = ii
                central_mol = rct
            else:
                current_central_reacting_atoms = len(reacting_atoms_reactants[central_index])
                if len(reacting_atoms) > current_central_reacting_atoms:
                    central_index = ii
                    central_mol = rct
                elif reacting_atoms == current_central_reacting_atoms:
                    if len(rct) > len(central_mol):
                        central_index = ii
                        central_mol = rct

        # Build complex around central_mol
        # TODO: you are here

        

    if len(pros) == 1:
        exit_complexes = pros
    else:
        pass